In [1]:
import pandas as pd
import requests
import urllib3
from datetime import date
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
auth_url = "https://www.strava.com/oauth/token"
athlete_url = "https://www.strava.com/api/v3/athlete"

In [22]:
# Get my user info for payload

user_info_file = 'my_info.csv'

try: my_info = pd.read_csv(user_info_file) 
except OSError: 
    print(f'Could not open {user_info_file} required to access client_id, client_secret, and refresh token')

In [12]:
payload = {
    'client_id': my_info['client_id'][0],
    'client_secret': my_info['client_secret'][0],
    'refresh_token': my_info['refresh_token'][0],
    'grant_type': "refresh_token",
    'f': 'json'
}

In [13]:
# Get distance run from strava (year-to-date)

res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']

header = {'Authorization': 'Bearer ' + access_token}
param = {'per_page': 200, 'page': 1}
my_dataset = requests.get(athlete_url, headers=header, params=param).json()

id_ = my_dataset['id']
stats_url = f"https://www.strava.com/api/v3/athletes/{id_}/stats"

activity_stats = requests.get(stats_url, headers=header, params=param).json()

ytd_run_totals = activity_stats['ytd_run_totals']
ytd_run_dist = ytd_run_totals['distance']/1000

print(f'Distance run this year: {round(ytd_run_dist)} km')

Distance run this year: 591 km


In [14]:
# Calculate how many days have passed this year

year = date.today().year
month = date.today().month
day = date.today().day

num_days_passed = date(year,month,day) - date(year,1,1)
num_days_passed = num_days_passed.days + 1

print(f'Number of days passed: {num_days_passed}')

Number of days passed: 132


In [15]:
# Calculate if on track

year_goal = 1600 # 1600 km = 1000 miles
avg_dist_required = round(year_goal/365,2) # Average distance per day to meet goal

avg_dist_current = round(ytd_run_dist/num_days_passed,2)

print(f'Your goal is {year_goal} km for the year')

print('You are: ', end = '')
if avg_dist_current == avg_dist_required:
    print('On track')
elif avg_dist_current > avg_dist_required:
    print('Ahead')
else:
    print('Behind')

print(f'Current average distance per day: {avg_dist_current} km, resulting in {round(avg_dist_current*365)} km')
print(f'Required average distance per day: {avg_dist_required} km')



Your goal is 1600 km for the year
You are: Ahead
Current average distance per day: 4.47 km, resulting in 1632 km
Required average distance per day: 4.38 km


In [102]:
# Calculate what is required to catch up 

catchup_time = 1 # In days

days_inc_catchup = num_days_passed + catchup_time

required_catchup_dist = round(catchup_time*avg_dist_required + (avg_dist_required - avg_dist_current)*num_days_passed)

print(f'To get back on track you need to run {required_catchup_dist} km in the next {catchup_time} days')

To get back on track you need to run 15 km in the next 1 days
